## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros

model_name = "refined_transactions_addresses_models"

# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=[model_name],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-14 22:09:15 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=46924
2024-12-14 22:09:15 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=46924
2024-12-14 22:09:15 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=46924
2024-12-14 22:09:15 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=46924
2024-12-14 22:09:15 [info     ] prepared 1 input batches.      filename=bydate.py lineno=96 process=46924


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model=model_name,
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-14 22:09:15 [info     ] Executing model function...    filename=testutils.py lineno=220 process=46924
2024-12-14 22:09:15 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=46924
2024-12-14 22:09:16 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53 process=46924
2024-12-14 22:09:16 [info     ] duckdb dataset='ingestion/blocks_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=46924
2024-12-14 22:09:18 [info     ] registered view: 'ingestion_blocks_v1' using 1 parquet paths filename=client.py lineno=53 process=46924
2024-12-14 22:09:18 [info     ] duckdb dataset='ingestion/logs_v1' using 1/22 parquet paths, first path is gs://

┌────────────────────────────────────────────────┐
│                      name                      │
│                    varchar                     │
├────────────────────────────────────────────────┤
│ daily_address_summary                          │
│ daily_transactions_agg_tx_from_tx_to_method    │
│ daily_transactions_agg_tx_from_tx_to_method_v1 │
│ daily_transactions_agg_tx_to                   │
│ daily_transactions_agg_tx_to_method            │
│ daily_transactions_agg_tx_to_method_v1         │
│ daily_transactions_agg_tx_to_v1                │
│ event_emitting_transactions                    │
│ event_emitting_transactions_v1                 │
│ ingestion_blocks_v1                            │
│ ingestion_logs_v1                              │
│ ingestion_transactions_v1                      │
│ logs_topic0_filters                            │
│ refined_transactions_fees                      │
│ refined_transactions_fees_v1                   │
│ summary_v1                   

## Verify model results - Transactions

In [3]:
duckdb_client.sql("SELECT * FROM daily_transactions_tx_from_tx_to_method_v1 ORDER BY count_transactions DESC LIMIT 10")

┌────────────┬─────────┬──────────┬─────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬────────────────────┬────────────────────────────┬─────────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬───────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────┬────────────────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────────────┬────────────────────────┬────────────────────────────────┬────────────────────────────────────┬────────────────────────────┬─────────────────────────────────┬───────────────────────┬──────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬────────────────────┐
│     dt     │  chain  │ chain_id │ network │                from_address       

In [4]:
duckdb_client.sql("SELECT * FROM daily_transactions_tx_to_method_v1 ORDER BY count_transactions DESC LIMIT 10")

┌────────────┬─────────┬──────────┬─────────┬────────────────────────────────────────────┬────────────┬────────────────────┬────────────────────────────┬─────────────────────┬─────────────────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬───────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────┬────────────────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────────────┬────────────────────────┬────────────────────────────────┬────────────────────────────────────┬────────────────────────────┬─────────────────────────────────┬───────────────────────┬──────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬────────────────────┐
│     dt     │  chain  │ chain_id │ network │                 to_address                 │ meth

In [5]:
duckdb_client.sql("SELECT * FROM daily_transactions_tx_to_v1 ORDER BY count_transactions DESC LIMIT 10")

┌────────────┬─────────┬──────────┬─────────┬────────────────────────────────────────────┬────────────────────┬────────────────────────────┬─────────────────────┬─────────────────────────────┬─────────────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬───────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────┬────────────────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────────────┬────────────────────────┬────────────────────────────────┬────────────────────────────────────┬────────────────────────────┬─────────────────────────────────┬───────────────────────┬──────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬────────────────────┐
│     dt     │  chain  │ chain_id │ network │                 to_address          

In [ ]:
# Check System tx identification
duckdb_client.sql("SELECT * FROM daily_transactions_tx_to_v1 ORDER BY count_transactions DESC LIMIT 10")

┌────────────┬─────────┬──────────┬─────────┬────────────────────────────────────────────┬────────────────────┬────────────────────────────┬─────────────────────┬─────────────────────────────┬─────────────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬───────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────┬────────────────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────────────┬────────────────────────┬────────────────────────────────┬────────────────────────────────────┬────────────────────────────┬─────────────────────────────────┬───────────────────────┬──────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬─────────────────────────┬────────────────────┐
│     dt     │  chain  │ chain_id │ network │                 to_address          

### Check the data output size

In [8]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT to_address) AS num_tos, SUM(count_transactions) AS num_txs FROM daily_transactions_tx_from_tx_to_method_v1")

┌──────────┬─────────┬─────────┐
│ num_rows │ num_tos │ num_txs │
│  int64   │  int64  │ int128  │
├──────────┼─────────┼─────────┤
│     5885 │    1374 │   31773 │
└──────────┴─────────┴─────────┘

In [9]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT to_address) AS num_tos, SUM(count_transactions) AS num_txs FROM daily_transactions_tx_to_method_v1")

┌──────────┬─────────┬─────────┐
│ num_rows │ num_tos │ num_txs │
│  int64   │  int64  │ int128  │
├──────────┼─────────┼─────────┤
│     2327 │    1374 │   31773 │
└──────────┴─────────┴─────────┘

In [10]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT to_address) AS num_tos, SUM(count_transactions) AS num_txs FROM daily_transactions_tx_to_v1")

┌──────────┬─────────┬─────────┐
│ num_rows │ num_tos │ num_txs │
│  int64   │  int64  │ int128  │
├──────────┼─────────┼─────────┤
│     1374 │    1374 │   31773 │
└──────────┴─────────┴─────────┘

### Get table schema

In [11]:
duckdb_client.sql("DESCRIBE daily_transactions_tx_to_v1")


┌────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│            column_name             │  column_type   │  null   │   key   │ default │  extra  │
│              varchar               │    varchar     │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                 │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                              │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                           │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                            │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ to_address                         │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ count_transactions                 │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ count_transactions_success         │ B